In [1]:
!pip install -q langchain==0.0.219 openai==0.27.8

# !pip install -q transformers==4.29.2 sentencepiece==0.1.99 accelerate==0.19.0 bitsandbytes==0.39.0

!pip install -q python-dotenv==1.0.0

!pip install -q pandas==1.5.3
!pip install -q unstructured==0.7.12
!pip install -q wikipedia==1.4.0
!pip install -q pypdf==3.12.0
!pip install -q jq==1.4.1
!pip install -q nltk==3.8.1
!pip install -q tiktoken==0.4.0
!pip install -q transformers==4.29.2 sentencepiece==0.1.99
!pip install -q sentence-transformers==2.2.2
!pip install -q cohere==4.11.2
!pip install -q faiss-cpu==1.7.4
!pip install -q chromadb==0.3.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 78.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 1.1 MB/s eta 0:00

In [2]:
from dotenv import dotenv_values
env_values = dotenv_values("./app.env")
openai_api_key = env_values['OPEN_API_KEY']

In [104]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.9, openai_api_key=openai_api_key)
llm2 = OpenAI(model_name="gpt-3.5-turbo",temperature=0.9, openai_api_key=openai_api_key)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
from langchain.document_loaders import WikipediaLoader
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


In [17]:
query_1 = "Nikola Tesla"
query_2 = "Thomas Edison"

text_document_1 = WikipediaLoader(query=query_1,doc_content_chars_max=20000,load_max_docs = 1).load()
text_document_2 = WikipediaLoader(query=query_2,doc_content_chars_max=20000,load_max_docs= 1).load()

In [18]:
text_document_2 =  text_document_2[0].page_content
text_document_1 =  text_document_1[0].page_content

In [19]:
documnets = [text_document_1, text_document_2]
metadata = [{"document":query_1},{"document" : query_2}]

In [20]:
from langchain.text_splitter import NLTKTextSplitter
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
text =  NLTKTextSplitter(chunk_size=300, chunk_overlap=20)
text = text.create_documents(documnets, metadatas=metadata )

In [26]:
len(text)

175

In [27]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [28]:
#embeddings =  embeddings.embed_documents(text)

vector =  Chroma.from_documents(text, embeddings,persist_directory="./chroma")

## Suff QNA

In [29]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")

In [86]:
promp = """
Answer the following question perfectly using provided context ,
If you can't find an answer of the question just diplay  I am stupid,
"#### Context" ,
{context},
""
"#### Question"
{question},
""
"####" Answer:
""
"""

In [87]:
temp = PromptTemplate(template=promp, input_variables=["context", "question"])

In [78]:
question = "when and where the father of tesla died"

In [79]:
similar = vector.similarity_search(query = question , k = 4 )

In [80]:
output = chain({"input_documents": similar, "question": question}, return_only_outputs=True)

In [81]:
print(output)

{'output_text': "Tesla's father, Milutin Tesla, died on 17 April 1879 in Smiljan."}


## Map reduce

In [88]:
promp_2 = """
"Given intermediate contexts for a question, generate a final answer.",
"If the answer is not contained in the intermediate contexts, say 'NO ANSWER IS AVAILABLE'",
"#### summaries" ,
{summaries},
"",
"#### Question"
{question},
"",
"#### Final Answer ",
""
"""

In [89]:
temp_2  = PromptTemplate(
    template=promp_2,
    input_variables=['summaries', 'question'])

In [95]:
map_reduce_chain = load_qa_chain(llm, chain_type="map_reduce",
                                 return_intermediate_steps=True,
                                 question_prompt=temp,
                                 combine_prompt=temp_2)

In [97]:
final_answer = map_reduce_chain({"input_documents": similar, "question": question}, return_only_outputs=True)


In [98]:
final_answer

{'intermediate_steps': ["Tesla's father died on 17 April 1879 in Gospić.",
  'His father, Milutin Tesla, died in 1879. The location of his death is not provided in the context.',
  "Tesla's father died in New York City in January 1943.",
  'I am stupid'],
 'output_text': 'NO ANSWER IS AVAILABLE'}

## Refine

In [106]:
initial_qna_template = "\n".join([
    "Answer the following question using the provided text only.",
    "If answer is not available. Say 'No answer for this context'",
    "### Context:",
    "{context_str}",
    "",
    "### Question:",
    "{question}",
    "### Answer:",
])

initial_qna_prompt = PromptTemplate(
    template=initial_qna_template,
    input_variables=['context_str', 'question']
)

In [107]:
refine_qna_template = "\n".join([
    "Refine the existing answer, if required, with the following context.",
    "If answer is not available. Say 'No answer for this context'",
    "### Context",
    "{context_str}",
    "",
    "### Existing Answer:",
    "{existing_answer}",
    "",
    "### Question:",
    "{question}",
    "",
    "### Refined Answer:",
])

refine_qna_prompt = PromptTemplate(
    template=refine_qna_template,
    input_variables=['context_str', 'existing_answer', 'question']
)

In [108]:
refine_chain = load_qa_chain(
    llm,
    chain_type="refine",
    question_prompt=initial_qna_prompt,
    refine_prompt=refine_qna_prompt,
    return_intermediate_steps=True,
)

In [109]:
final_answer = refine_chain({"input_documents": similar, "question": question}, return_only_outputs=True)
final_answer

{'intermediate_steps': ["Tesla's father died on 17 April 1879 in Gospić.",
  "Tesla's father, Milutin Tesla, died on April 17, 1879, in Gospić.",
  "Tesla's father, Milutin Tesla, died on April 17, 1879, in Gospić, Croatia.",
  "Tesla's father, Milutin Tesla, died on April 17, 1879, in Gospić, Croatia, after Tesla finished a four-year term in three years and graduated in 1873."],
 'output_text': "Tesla's father, Milutin Tesla, died on April 17, 1879, in Gospić, Croatia, after Tesla finished a four-year term in three years and graduated in 1873."}